In [6]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

import glob
import sys
import dask

from CASutils import lensread_utils as lens
from CASutils import readdata_utils as read
from math import nan

In [7]:
topdir="/glade/campaign/cesm/collections/CESM2-SF/timeseries/"

In [8]:
landfrac = xr.open_dataset('LANDFRAC_LENS2.nc')
landfrac = landfrac.landfrac

In [9]:
landfrac = xr.where(landfrac > 0.5, 1, nan)

In [10]:
memstr = [ str(imem).zfill(3) for imem in np.arange(1,4,1) ]

In [11]:
import dask
from dask_jobqueue import PBSCluster
from dask.distributed import Client

cluster = PBSCluster(
    cores = 1,
    memory = '50GB',
    processes = 1,
    queue = 'casper',
    local_directory = '$TMPDIR',
    resource_spec = 'select=1:ncpus=1:mem=20GB',
    project='P04010022',
    walltime='02:00:00',
    interface='ib0')

# scale up
cluster.scale(20)

# change your urls to the dask dashboard so that you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})

# Setup your client
client = Client(cluster)

/glade/u/home/islas/miniconda3/envs/ecpaperenv/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46463 instead
  http_address["port"], self.http_server.port


In [12]:
cluster

PBSCluster('tcp://10.12.206.63:36167', workers=0, threads=0, memory=0 B)

In [13]:
#client.close()

In [15]:
client

Client Scheduler: tcp://10.12.206.63:36167 Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/islas/proxy/46463/status,Cluster Workers: 4 Cores: 4 Memory: 85.90 GB


In [16]:
for imem in memstr:
    print(imem)
    filelist = sorted(glob.glob(topdir+"b.e21.BHISTcmip6.f09_g17.CESM2-SF-xAER."+imem+
                                "/lnd/proc/tseries/day_1/*.FSNO.*.nc"))
    dat = xr.open_mfdataset(filelist, concat_dim=['time'], coords='minimal')[['FSNO','time_bounds']]
    dat = read.fixcesmtime(dat, timebndsvar='time_bounds')
    dat['lon'] = landfrac.lon ; dat['lat'] = landfrac.lat
    
    lons=[286,174,95,213]
    lats=[59,68,73,61]
    locations=[]
    for i in np.arange(0,len(lons),1):
        loc = dat.sel(lon=lons[i],lat=lats[i], method='nearest')
        locations.append(loc)
    locations = xr.concat(locations, dim='point')
    
    locations = locations.where( ((locations.time.dt.month == 6) | (locations.time.dt.month == 7) | (locations.time.dt.month == 8)), drop=True)
    locations.to_netcdf("/glade/scratch/islas/python/singleforcing/DATA_SORT/FSNO/FSNO_JJA_locations_XAER2_"+imem+".nc")

001
002
003


In [17]:
#client.close()